# <div align='center'> 分类任务测试 </div>

In [ ]:
%reload_ext autoreload
%autoreload 2

In [2]:
backbones = [
    'resnet18',           # 0
    'resnet50',           # 1
    'densenet121',        # 2
    'mobilenet_v2',       # 3
    'squeezenet1_0',      # 4
    'squeezenet1_1',      # 5
    'shufflenet_v2_x0_5', # 6
    'shufflenet_v2_x1_0', # 7
]

batch_sizes = [8, 16, 32, 64]
input_sizes = [28, 32, 64, 128, 256]

## 任务

In [8]:
from k12libs.utils.nb_easy import k12ai_run_project
project = k12ai_run_project(
    debug=True, tb_port=9002,
    framework='k12cv', task='cls',
    network=backbones[3], dataset='mnist')

Box(children=(Box(children=(VBox(children=(Tab(children=(VBox(children=(HBox(children=(Text(value='cls', conti…

## 压测 (P40 跑10个任务)

In [ ]:
from k12libs.utils.nb_easy import k12ai_train_execute
keys = k12ai_train_execute(
    framework='k12cv', task='cls',
    network=backbones[-1], dataset='rmnist',
    batchsize=32, inputsize=32, epoch_num=20, run_num=10)